In [16]:
import glob
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

from tensorflow import keras

from sklearn.model_selection import train_test_split

In [17]:
instrument_Names = ['guitar','sexophone']

In [18]:
train = []
target = []

In [19]:
for name in instrument_Names:
    for image in sorted(glob.glob('../../Data/%sresize/*.jpg'%name)):
        img = Image.open(image)
        img = img.convert('L')
        train.append(img)
        target.append(name)

# train,test 만들기

In [20]:
x_train, x_test, y_train, y_test = train_test_split(
    train, target, test_size=0.2, random_state=42
)

In [38]:
trainData = np.zeros(len(x_train) * 100*100, dtype=np.int32).reshape(len(x_train), 100, 100)
testData = np.zeros(len(x_test) * 100*100, dtype=np.int32).reshape(len(x_test), 100, 100)

In [39]:
i = 0
for image in x_train:
    img = np.array(image, dtype=np.int32)
    trainData[i,:,:] = img
    i += 1

i = 0
for image in x_test:
    img = np.array(image, dtype=np.int32)
    testData[i,:,:] = img
    i += 1

In [40]:
print(trainData.shape)
print(testData.shape)

(32, 100, 100)
(8, 100, 100)


In [46]:
#data의 정규화
trainData=trainData.astype('float32')/255
testData=testData.astype('float32')/255

trainData=trainData.reshape((len(trainData),100*100))
testData=testData.reshape((len(testData),100*100))

In [47]:
print(trainData.shape)
print(testData.shape)

(32, 10000)
(8, 10000)


### Tensorflow Model 만들기

In [54]:
# MNIST Data를 입력으로
InputModel=keras.layers.Input(shape=(100*100))

#EncoderLayer는 3차원으로 축소된 입력값
EncodedLayer=keras.layers.Dense(32,activation='relu')(InputModel)

# DecoderLayer는 MNIST 차원으로 복원된 값
DecoderLayer=keras.layers.Dense(100*100,activation='sigmoid')(EncodedLayer)

# AutoEncoderModel은 입력값과 복원값을 최소화 하는 과정
AutoencoderModel=keras.models.Model(InputModel,DecoderLayer)

In [55]:
AutoencoderModel.compile(optimizer='adam',loss='binary_crossentropy')

In [56]:
history=AutoencoderModel.fit(trainData,trainData,
                             batch_size=256,
                             epochs=50,
                             shuffle=True,
                             validation_data=(testData,testData)
                             )

Epoch 1/50
1/1 [==============================] - 1s 889ms/step - loss: 0.6931 - val_loss: 0.6927
Epoch 2/50
1/1 [==============================] - 0s 53ms/step - loss: 0.6927 - val_loss: 0.6922
Epoch 3/50
1/1 [==============================] - 0s 56ms/step - loss: 0.6922 - val_loss: 0.6917
Epoch 4/50
1/1 [==============================] - 0s 57ms/step - loss: 0.6917 - val_loss: 0.6912
Epoch 5/50
1/1 [==============================] - 0s 54ms/step - loss: 0.6912 - val_loss: 0.6906
Epoch 6/50
1/1 [==============================] - 0s 47ms/step - loss: 0.6906 - val_loss: 0.6900
Epoch 7/50
1/1 [==============================] - 0s 59ms/step - loss: 0.6900 - val_loss: 0.6893
Epoch 8/50
1/1 [==============================] - 0s 50ms/step - loss: 0.6893 - val_loss: 0.6884
Epoch 9/50
1/1 [==============================] - 0s 47ms/step - loss: 0.6884 - val_loss: 0.6874
Epoch 10/50
1/1 [==============================] - 0s 47ms/step - loss: 0.6874 - val_loss: 0.6861
Epoch 11/50
1/1 [===========